In [ ]:
'''
measuring the importance of each clinical feature for classifying between MS and HC.
The most discriminative features are first selected using the Mann-Whitney U test, a statistical method that identifies significant differences between the two groups.
The SHAP (SHapley Additive exPlanations) method is then used to determine feature importance.
@author: Asieh Soltanipour, Asieh.soltanipour1365@gmail.com
'''
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from matplotlib.collections import LineCollection
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
from operator import itemgetter
from decimal import Decimal
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay, roc_curve, precision_recall_curve,confusion_matrix,RocCurveDisplay,accuracy_score,roc_auc_score,auc
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold,StratifiedKFold
from sklearn.manifold import TSNE
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def preparing(x, y):

     data  = []
     label = []
     for i in x:
         for j in range(len(x[i])):
             data.append(x[i][j])
             label.append(y[i])

     data = np.reshape(data, np.shape(data))
     #normalizing data
     for k in range(data.shape[1]):
         q=np.where(data[:,k] != -1)[0]
         data[q,k]=(data[q,k]-data[q,k].min())/(data[q,k].max()-data[q,k].min())
     return data, label


In [ ]:
def metrics_calculation(y_valid, y_pred, y_prob):

    #####################################################
    #Get the confusion matrix
    #####################################################
    ROC_AUC = roc_auc_score(y_valid, y_prob)

    f1 = metrics.f1_score(y_valid, y_pred, average='weighted')
    precision, recall, thresholds = precision_recall_curve(y_valid, y_prob)
    P_R_AUC = auc(recall, precision)
    cm = confusion_matrix(y_valid, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    class_acc = cm.diagonal()
    Specificity = cm[0,0]/(cm[0,0]+cm[0,1])
    Sensitivity = cm[1,1]/(cm[1,0]+cm[1,1])
    Precision   = cm[1,1]/(cm[0,1]+cm[1,1])
    return Specificity, Sensitivity, Precision, f1, ROC_AUC, P_R_AUC, class_acc, cm

In [ ]:

def fold_curves(ax, model, x_valid, y_valid, fold_number, mean_fpr, tprs=[], aucs=[]):
    ############ ROC Curve

    viz = RocCurveDisplay.from_estimator(
        model,
        x_valid,
        y_valid,
        name="ROC Curve fold {}".format(fold_number),
        alpha=0.3,
        lw=1,
        ax=ax,
    )
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    return tprs, aucs

In [ ]:
'''
loading training SLO features:
features_SLO is a dictionary in which its keys is the number of subjects and the corresponding value for each key in an array with size of (number of images per subject X number of features).
labels_SLO is a dictionary in which its  key defined the nember of subject and its corresponding value is lable of each subject(MS and HC)
'''
features_SLO=pickle.load(open('/content/drive/MyDrive/'+'features_SLO'+'.pkl', 'rb'))
labels_SLO=pickle.load(open('/content/drive/MyDrive/'+'labels_SLO'+'.pkl', 'rb'))

In [ ]:
'''
applying a statistical-based filter method to our dataset for feature selection (FS),
dividing features_SLO into MS and HC groups and then computing the mean values of each feature across all IR-SLO images for individual subjects.
Given the non-normal distribution of clinical features, we employed the Mann-Whitney U test to identify significant features differentiating these groups
'''
HC_features=[]
MS_features=[]
for i in list(labels_SLO.keys()):
  #HC
  if labels_SLO[i] == 0:
    HC_features.append(np.mean(features_SLO[i],axis=0))
  if labels_SLO[i] == 1:
    MS_features.append(np.mean(features_SLO[i],axis=0))
HC_features=np.array(HC_features)
MS_features=np.array(MS_features)

from scipy.stats import mannwhitneyu
statistic, p_value = mannwhitneyu(HC_features, MS_features, alternative='two-sided')
# Using a significance level set at p < 0.05 to identify discriminating features between HC and MS individuals
indices = np.where(p_value < 0.05)[0]


In [ ]:
'''
Before building training and validation data  sets using k-fold cross-validation , it is necessary to creat an internal test.
To create the internal test dataset, you can use a subject-wise spliting or a stochastic matching method based on age and gender.

In the latter method, initially, 20% of subjects with MS were randomly chosen and designated as the internal test dataset. For each selected MS case, an HC patient with the closest
  age and the same gender was also included in the age-gender matching test dataset.

after creating internal test, you should consider features with p_value<0.05

'''
# Separate subjects by label
hc_subjects = [subj for subj, label in labels_SLO.items() if label == 0]
ms_subjects = [subj for subj, label in labels_SLO.items() if label == 1]

# Define split ratio (e.g., 80% training, 20% test)
train_ratio = 0.8

# Shuffle and split each group into training and test sets
hc_train, hc_test = train_test_split(hc_subjects, train_size=train_ratio, random_state=42)
ms_train, ms_test = train_test_split(ms_subjects, train_size=train_ratio, random_state=42)

# Combine training and test subjects
train_subjects = hc_train + ms_train
test_subjects = hc_test + ms_test

# Prepare training and test data and labels
train_data = {subj: features_SLO[subj] for subj in train_subjects}
internal_test_data = {subj: features_SLO[subj] for subj in test_subjects}

train_labels = {subj: labels_SLO[subj] for subj in train_subjects}
internal_test_labels = {subj: labels_SLO[subj] for subj in test_subjects}

test,label_test = preparing(internal_test_data,internal_test_labels)
FS_test = test[:, indices]

In [ ]:
'''
Applying k-fold cross-validation for spliting train_data into training and validation  and considering features with p_value<0.05:

'''

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold (n_splits = 5, shuffle = True, random_state = None)
nfold = 5  #please enter number of folds
kf_nfold = StratifiedKFold(n_splits=nfold, random_state=None, shuffle=True)
n = 0
x_train_folds={}
x_valid_folds={}
y_train_folds={}
y_valid_folds={}
for train_index, val_index in kf_nfold.split(train_data,list(train_labels.values())):

    train_index, val_index = next (skf.split (train_data, list(train_labels.values())))
    x_train = {i: train_data[list(train_data.keys())[i]]  for i in train_index}
    x_valid = {i: train_data[list(train_data.keys())[i]]  for i in val_index}
    y_train = {i: train_labels[list(train_labels.keys())[i]] for i in train_index}
    y_valid = {i: train_labels[list(train_labels.keys())[i]] for i in val_index}
    x_train,y_train = preparing(x_train,y_train)
    x_valid,y_valid = preparing(x_valid,y_valid)
    FS_x_train = x_train[:, indices]
    FS_x_valid = x_valid[:, indices]
    x_train_folds[n]=FS_x_train
    y_train_folds[n]=y_train
    x_valid_folds[n]=FS_x_valid
    y_valid_folds[n]=y_valid
    n = n+1



'''loading external test data :'''
features_SLO__external_test=pickle.load(open('/content/drive/MyDrive/'+'features_SLO_external_test'+'.pkl', 'rb'))
labels_SLO_external_test=pickle.load(open('/content/drive/MyDrive/'+'labels_SLO_external_test'+'.pkl', 'rb'))
###applyting prepreing and selectiong feature with p_value<0.05
test_external,label_test_external = preparing(features_SLO__external_test,labels_SLO_external_test)
FS_test_external = test_external[:, indices]


In [ ]:
'''
 SVM classifier to classify IR-SLO images into 2 classes: Ms and HC
 we updated SVM hyperparameters for linear, RBF, sigmoid and poly kernels by using Optuna hyperparameters software
 '''
acc_lin, acc_rbf, acc_sig  = [],[],[]
sp_lin, sp_rbf, sp_sig  = [],[],[]
se_lin, se_rbf, se_sig  = [],[],[]
pr_lin, pr_rbf, pr_sig  = [],[],[]
f1_lin, f1_rbf, f1_sig   = [],[],[]
auc_lin, auc_rbf,auc_sig   = [],[],[]
pr_auc_lin, pr_auc_rbf, pr_auc_sig   = [],[],[]
class_acc_rbf=np.zeros((2))
number_class=2
confusion_matrix_rbf=np.zeros((number_class, number_class))


for n_fold in range(5):

    x= x_train_folds[n_fold]
    y= y_train_folds[n_fold]

    linear = svm.SVC(class_weight =None, probability=True,C= 4025.7688422856236,kernel='linear').fit(x,y)
    rbf = svm.SVC(class_weight =None, probability=True,C= 7072.776159189032,gamma=3.9196303352470423 ,kernel='rbf').fit(x,y)
    sig = svm.SVC(class_weight =None, probability=True,C=7293.275917107514,gamma=0.027443435885140987 ,kernel='sigmoid').fit(x,y)

    ACC_l = linear.score(x_valid_folds[n_fold],y_valid_folds[n_fold])
    ACC_r = rbf.score(x_valid_folds[n_fold],y_valid_folds[n_fold])
    ACC_s = sig.score(x_valid_folds[n_fold],y_valid_folds[n_fold])


    linear_pred_val = linear.predict(x_valid_folds[n_fold])
    rbf_pred_val = rbf.predict(x_valid_folds[n_fold])
    sig_pred_val = sig.predict(x_valid_folds[n_fold])

    linear_proba = linear.predict_proba(x_valid_folds[n_fold])[:,1]
    rbf_proba    = rbf.predict_proba(x_valid_folds[n_fold])[:,1]
    sig_proba    = sig.predict_proba(x_valid_folds[n_fold])[:,1]

    SP_l, SE_l, PR_l, f1_l, ROC_AUC_l, P_R_AUC_l, class_acc_l, cm_l = metrics_calculation(y_valid_folds[n_fold], linear_pred_val, linear_proba)
    SP_r, SE_r, PR_r, f1_r, ROC_AUC_r, P_R_AUC_r, class_acc_r, cm_r = metrics_calculation(y_valid_folds[n_fold], rbf_pred_val, rbf_proba)
    SP_s, SE_s, PR_s, f1_s, ROC_AUC_s, P_R_AUC_s, class_acc_s, cm_s = metrics_calculation(y_valid_folds[n_fold], sig_pred_val, sig_proba)

    class_acc_rbf  = np.add(class_acc_rbf,class_acc_r)
    confusion_matrix_rbf = np.add(confusion_matrix_rbf,cm_r)

    acc_lin.append(ACC_l)
    acc_rbf.append(ACC_r)
    acc_sig.append(ACC_s)

    sp_lin.append(SP_l)
    sp_rbf.append(SP_r)
    sp_sig.append(SP_s )

    se_lin.append(SE_l)
    se_rbf.append(SE_r)
    se_sig.append(SE_s)

    pr_lin.append(PR_l)
    pr_rbf.append(PR_r)
    pr_sig.append(PR_s)

    f1_lin.append(f1_l)
    f1_rbf.append(f1_r)
    f1_sig.append(f1_s)

    auc_lin.append(ROC_AUC_l)
    auc_rbf.append(ROC_AUC_r)
    auc_sig.append(ROC_AUC_s)

    pr_auc_lin.append(P_R_AUC_l)
    pr_auc_rbf.append(P_R_AUC_r)
    pr_auc_sig.append(P_R_AUC_s)


print('****svm with linear***** :')
print(f' acc : {np.mean(acc_lin)}, sp : {np.mean(sp_lin)}, se : {np.mean(se_lin)}, pr : {np.mean(pr_lin)},f1 : {np.mean(f1_lin)}, auc : {np.mean(auc_lin)}, pr_auc : {np.mean(pr_auc_lin)} ')

print('****svm with rbf***** :')
print(f' acc : {np.mean(acc_rbf)}, sp : {np.mean(sp_rbf)}, se : {np.mean(se_rbf)}, pr : {np.mean(pr_rbf)}, f1 : {np.mean(f1_rbf)}, auc : {np.mean(auc_rbf)}, pr_auc : {np.mean(pr_auc_rbf)}')
class_acc_rbf  = class_acc_rbf/5
print(f' class ACC : {class_acc_rbf}')
confusion_matrix_rbf  = confusion_matrix_rbf/5
print(f' confusion_matrix : {confusion_matrix_rbf}')

print('****svm with sig***** :')
print(f' acc : {np.mean(acc_sig)}, sp : {np.mean(sp_sig)}, se : {np.mean(se_sig)}, pr : {np.mean(pr_sig)}, f1 : {np.mean(f1_sig)}, auc : {np.mean(auc_sig)}, pr_auc : {np.mean(pr_auc_sig)} ')


target_names = ['Normal' , 'MS']
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_rbf, display_labels=target_names)
disp.plot()
plt.title('SVM classifier (kernel : RBF)')


In [ ]:
'''using Random Forest classifier :
'''
acc_rf,sp_rf, se_rf, pr_rf,f1_rf, auc_rf, pr_auc_rf, tprs_rf, aucs_rf, y_pred_rf = [],[],[],[],[],[],[],[],[],[]
class_acc_rf=np.zeros((2))
number_class=2
confusion_matrix_rf=np.zeros((number_class, number_class))

for n_fold in range(5):

    x=x_train_folds[n_fold]
    y=y_train_folds[n_fold]
    model= RandomForestClassifier(
        random_state=42,max_depth=10,n_estimators=147,min_samples_split=10,min_samples_leaf=2,max_features='sqrt',criterion='gini'
        )
    model.fit(x,y)
    valid=x_valid_folds[n_fold]
    ACC_rf=model.score(valid,y_valid_folds[n_fold])
    rf_pred_val = model.predict(valid)
    rf_proba = model.predict_proba(valid)[:,1]
    SP_rf, SE_rf, PR_rf, f1, ROC_AUC_rf, P_R_AUC_rf, class_acc_rf0, cm_rf = metrics_calculation(y_valid_folds[n_fold], rf_pred_val, rf_proba)

    acc_rf.append(ACC_rf)
    sp_rf.append(SP_rf)
    se_rf.append(SE_rf)
    pr_rf.append(PR_rf)
    f1_rf.append(f1)
    auc_rf.append(ROC_AUC_rf)
    pr_auc_rf.append(P_R_AUC_rf)
    class_acc_rf  = np.add(class_acc_rf,class_acc_rf0)
    confusion_matrix_rf = np.add(confusion_matrix_rf,cm_rf)


print(f'acc:{np.mean(acc_rf)}')
print(f'sp:{np.mean(sp_rf)}')
print(f'se:{np.mean(se_rf)}')
print(f'pr:{np.mean(pr_rf)}')
print(f'f1:{np.mean(f1_rf)}')
print(f'auc:{np.mean(auc_rf)}')
print(f'pr_auc:{np.mean(pr_auc_rf)}')

class_acc_rf  = class_acc_rf/5
print(f' class ACC : {class_acc_rf}')
confusion_matrix_rf  = confusion_matrix_rf/5
print(f' confusion_matrix : {confusion_matrix_rf}')
target_names = ['Normal' , 'MS']
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_rf, display_labels=target_names)
disp.plot()
plt.title('Random Forest Classifier')

In [ ]:
'''
XGBoost classifier:
'''
acc_XGBoost ,sp_XGBoost,se_XGBoost, pr_XGBoost,f1_XGBoost,auc_XGBoost,pr_auc_XGBoost,tprs_XGBoost,aucs_XGBoost,y_pred_XGBoost= [],[],[],[],[],[],[],[],[],[]
class_acc_xgb=np.zeros((2))
number_class=2
confusion_matrix_xgb=np.zeros((number_class, number_class))


param = {
    "verbosity": 0,
    "objective": "binary:logistic",
    "tree_method": "auto",
    "booster" : "gbtree",
    "lambda" :  0.6262819848273675,
    "alpha": 0.002417237009326353,
    "subsample": 0.6816657652094626,
    "colsample_bytree" :  0.3288592429227775,
    "max_depth" : 3,
    "min_child_weight" : 4,
    "eta":0.02168530523973617,
    "gamma":  0.01720205835153038,
    "grow_policy": "depthwise",
}


for n_fold in range(5):

    x=x_train_folds[n_fold]
    y=y_train_folds[n_fold]
    valid=x_valid_folds[n_fold]

    model= XGBClassifier(**param)
    model.fit(x,y)
    ACC_XGBoost=model.score(valid,y_valid_folds[n_fold])
    XGBoost_pred_val = model.predict(valid)
    XGBoost_proba = model.predict_proba(valid)[:,1]
    SP, SE, PR, f1, ROC_AUC, P_R_AUC, class_acc, cm = metrics_calculation(y_valid_folds[n_fold], XGBoost_pred_val, XGBoost_proba)

    acc_XGBoost.append(ACC_XGBoost)
    sp_XGBoost.append(SP)
    se_XGBoost.append(SE)
    pr_XGBoost.append(PR)
    f1_XGBoost.append(f1)
    auc_XGBoost.append(ROC_AUC)
    pr_auc_XGBoost.append(P_R_AUC)
    class_acc_xgb  = np.add(class_acc_xgb,class_acc)
    confusion_matrix_xgb = np.add(confusion_matrix_xgb,cm)

print(f'acc: {np.mean(acc_XGBoost)}')
print(f'sp: {np.mean(sp_XGBoost)}')
print(f'se: {np.mean(se_XGBoost)}')
print(f'pr: {np.mean(pr_XGBoost)}')
print(f'f: {np.mean(f1_XGBoost)}')
print(f'auc: {np.mean(auc_XGBoost)}')
print(f'pr_auc: {np.mean(pr_auc_XGBoost)}')
class_acc_xgb  = class_acc_xgb/5
print(f' class ACC : {class_acc_xgb}')
confusion_matrix_xgb  = confusion_matrix_xgb/5
print(f' confusion_matrix : {confusion_matrix_xgb}')


target_names = ['Normal' , 'MS']
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_xgb, display_labels=target_names)
disp.plot()
plt.title('XGBoost Classifier')

In [ ]:
'''
 computing feature importance using SHAP method:
'''
!pip install shap

In [ ]:
#list of the name of selected features by Mann-Whiteny U test
#list of manualand clinical features used:
features_name=[
         #features from segmented disc and cup:
        'Disc_width','Cup_width','Disc-Cup widths ratio',
         #features from segmented blood vessels in the whole IR-SLO images:
        'whole_vessel_width','whole_vessel_density','whole_fractal_dimension','disctance_measure_tortousity','squared_curvature_tortousity',
        'tortousity_density','whole_intensity_vessel',
         #features from segmented vessels in zone B and zone C:
        'vessel_width_Zone B','vessel_width_Zone C','fractal_dimension_zoneB','fractal_dimension_zoneC',
        'linear_regression_tortousity_ZoneB','linear_regression_tortousity_ZoneC',
        'disctance_measure_tortousity_ZoneB','disctance_measure_tortousity_ZoneC',
        'squared_curvature_tortousity_ZoneB','squared_curvature_tortousity_ZoneC',
        'tortousity_density_ZoneB','tortousity_density_Zonec',
        'vessel_density_zoneB','vessel_density_zoneC',
         ]
static_selected_features_name = [features_name[i] for i in list(indices)]



In [ ]:
import shap
from shap import Explainer, Explanation

def compute_SHAP_importance(x_train_folds,data_inpute,y_input,param,name_data):
    list_shap_values = list()
    importances_fold={}
    for n_fold in range(5):
         x=x_train_folds[n_fold]
         y=y_train_folds[n_fold]
         x_df=pd.DataFrame(x,columns=static_selected_features_name)
         model= XGBClassifier(**param,)
         model.fit(x_df,y)
         explainer = shap.TreeExplainer(model)
         if name_data == 'training':
            shape_values1=explainer(pd.DataFrame(x, columns=static_selected_features_name))
         elif name_data == 'validation':
            shape_values1=explainer(pd.DataFrame(data_inpute[n_fold], columns=static_selected_features_name))
         elif name_data == 'internal test':
            shape_values1=explainer(pd.DataFrame(data_inpute, columns=static_selected_features_name))
         else:
            shape_values1=explainer(pd.DataFrame(data_inpute, columns=static_selected_features_name))
         list_shap_values.append(shape_values1)

         importances=[]
         for i in range(shape_values1.values.shape[1]):
             importances.append(np.mean(np.abs(shape_values1.values[:, i])))
         importances_fold[n_fold]=importances

    #concatenating shap values for 5 folds:
    shap_t_values=np.concatenate((list_shap_values[0].values,list_shap_values[1].values,list_shap_values[2].values,
                             list_shap_values[3].values,list_shap_values[4].values),axis=0)
    shap_t_base=np.concatenate((list_shap_values[0].base_values,list_shap_values[1].base_values,list_shap_values[2].base_values,
                             list_shap_values[3].base_values,list_shap_values[4].base_values),axis=0)

    shap_t_data=np.concatenate((list_shap_values[0].data,list_shap_values[1].data,list_shap_values[2].data,
                             list_shap_values[3].data,list_shap_values[4].data),axis=0)
    ###
    '''
     as zone B and zone C were not defined in some IR-SLO images, those that do not containing OD,
     it is nessesary to normalize shap values by the number of and the number of MS images in which these zones have been defined
    '''
    if name_data == 'training':
         XX=np.concatenate((data_inpute[0],data_inpute[1],data_inpute[2],data_inpute[3],data_inpute[4]),axis=0)
         y_tt=y_input[0]+y_input[1]+y_input[2]+y_input[3]+y_input[4]
    elif name_data == 'validation':
         XX=np.concatenate((data_inpute[0],data_inpute[1],data_inpute[2],data_inpute[3],data_inpute[4]),axis=0)
         y_tt=y_input[0]+y_input[1]+y_input[2]+y_input[3]+y_input[4]
    else:
         xx=data_inpute
         y_tt=y_input[0]
    XX1=np.copy(XX)
    y_tt=np.array(y_tt)
    y_tt1=np.copy(y_tt)
    values1=np.copy(shap_t_values)
    p=np.where(XX1==-1)[0]# number of images in which these zones are not defined
    y_tt1=np.delete(y_tt1, list(p), 0)
    n_HC=len(np.where(np.array(y_tt1)==0)[0])
    n_MS=len(np.where(np.array(y_tt1)==1)[0])
   # normalizing shap values and observation
    for ii in range(XX.shape[1]):
          qq=np.where(XX[:,ii] != -1)[0]
          for jj in range(len(qq)):
            if y_tt[qq[jj]] == 1 :
              XX1[qq[jj],ii]=XX1[qq[jj],ii]/(n_MS/len(y_tt1))
              values1[qq[jj],ii]=values1[qq[jj],ii]/(n_MS/len(y_tt1))
            if y_tt[qq[jj]] ==0 :
              XX1[qq[jj],ii]=XX1[qq[jj],ii]/(n_HC/len(y_tt1))
              values1[qq[jj],ii]=values1[qq[jj],ii]/(n_HC/len(y_tt1))

    Shap_normalized = Explanation(values1, shap_t_base, shap_t_data , feature_names=static_selected_features_name)
    #ploting shap values
    #shap.summary_plot(values1,XX1,feature_names=features_name,plot_size=[10,8],show = True, class_names=target_names,max_display=len(features_name))
    #shap.plots.bar(Shap_normalized,max_display=len(features_name))
    Shap_sumation=Shap_normalized.sum(axis=0)
    #shap.plots.bar(Shap_sumation,max_display=len(features_name))
    ##

  # max_importance_shap : a list includng the order of important features cmputed by SHAP method
    importance_fold_shap_external=list(np.mean(np.abs(shap_t_values),axis=0))

    shap_importance[name_data]=importance_fold_shap_external

    return shap_importance, values1,Shap_normalized

shap_importance={}
evaluation_list=['training','validation','internal_test','external_test']

data=[x_train_folds,x_valid_folds,FS_test,FS_test_external]
y_data=[y_train_folds,y_valid_folds,label_test,label_test_external]
c=0
for i in evaluation_list:
    shap_importance, values1,Shap_normalized=compute_SHAP_importance(x_train_folds, data[c],y_data[c] ,param, i)
    c += 1


cmap = sns.diverging_palette(220, 10, as_cmap=True)
u=np.zeros((len(evaluation_list),len(static_selected_features_name)))
CC_heat_map=[shap_importance['training'], shap_importance['validation'], shap_importance['internal_test'], shap_importance['external_test']]

for i in range(u.shape[0]):
  u[i,:]=CC_heat_map[i]
df00=pd.DataFrame(index=evaluation_list,data=u,columns=static_selected_features_name)
fig, ax = plt.subplots(figsize=(16,4))
sns.heatmap(df00,cmap="Greens",linewidths=0.5,ax=ax,annot=True,fmt=".2f",annot_kws={"size": 11, },linewidth=0.5)